## Import Libraries and Load data | Day_Treatment_Rep

In [4]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np

In [5]:
df = pd.read_csv('../materials/process_csv/weight_color_data.csv')
df['Day'] = df['Label'].apply(lambda x: x.split('_')[0])
df['Temp'] = df['Label'].apply(lambda x: x.split('_')[1])
df.drop('Label', axis=1, inplace=True)
df.head()

,Weight,R_Mean,R_Std,G_Mean,G_Std,B_Mean,B_Std,H_Mean,H_Std,S_Mean,...,V_Mean,V_Std,L_Mean,L_Std,a_Mean,a_Std,b_Mean,b_Std,Day,Temp
0,158.9,202.031017,63.184791,206.361271,55.761869,195.552566,74.087194,48.603222,56.316379,26.942528,...,207.003862,55.870933,208.442289,55.784846,124.388767,7.205789,133.458801,10.995181,0,5
1,154.5,196.388049,57.669474,200.363846,50.615080,189.855627,68.516092,63.280031,57.690568,26.164496,...,201.372722,50.922278,203.303091,50.910815,124.570902,7.112593,133.317630,10.926267,1,5
2,149.7,196.586956,56.816229,200.405571,50.129378,190.606173,67.044048,63.870066,57.068227,24.570594,...,201.448391,50.424279,203.433086,50.298129,124.747403,6.862492,132.948486,10.475270,2,5
3,148.4,206.840822,58.391917,210.816657,51.437818,201.517561,68.725207,52.674892,54.528216,22.998771,...,211.541725,51.588476,212.927224,51.478033,124.804936,6.758229,132.644228,10.385137,3,5
4,147.5,207.421014,58.118789,211.122241,51.420555,202.135509,68.412750,53.727779,56.380803,22.434080,...,211.884094,51.545167,213.275613,51.336971,124.952920,6.614468,132.517304,10.255598,4,5


In [6]:
df.shape

(337, 21)

In [7]:
df.columns

Index(['Weight', 'R_Mean', 'R_Std', 'G_Mean', 'G_Std', 'B_Mean', 'B_Std',
       'H_Mean', 'H_Std', 'S_Mean', 'S_Std', 'V_Mean', 'V_Std', 'L_Mean',
       'L_Std', 'a_Mean', 'a_Std', 'b_Mean', 'b_Std', 'Day', 'Temp'],
      dtype='object')

## Training

In [8]:
import cv2
import numpy as np
import pandas as pd
from itertools import combinations
from lightgbm import LGBMRegressor
from xgboost import XGBRegressor
from sklearn.linear_model import LinearRegression, Ridge, Lasso, ElasticNet, HuberRegressor, QuantileRegressor
from sklearn.ensemble import RandomForestRegressor
from sklearn.preprocessing import PolynomialFeatures
from sklearn.pipeline import make_pipeline
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.preprocessing import StandardScaler
import logging

# Configure logging
logging.basicConfig(level=logging.INFO, format='%(asctime)s - %(levelname)s - %(message)s')

/home/j/Desktop/SeniorProject/kale_venv/lib/python3.12/site-packages/xgboost/core.py:265: FutureWarning: Your system has an old version of glibc (< 2.28). We will stop supporting Linux distros with glibc older than 2.28 after **May 31, 2025**. Please upgrade to a recent Linux distro (with glibc 2.28+) to use future versions of XGBoost.
Note: You have installed the 'manylinux2014' variant of XGBoost. Certain features such as GPU algorithms or federated learning are not available. To use these features, please upgrade to a recent Linux distro with glibc 2.28+, and install the 'manylinux_2_28' variant.
  warnings.warn(


In [9]:

base_features = {
    "L": ["L_Mean", "L_Std"],
    "a": ["a_Mean", "a_Std"],
    "b": ["b_Mean", "b_Std"],
    "H": ["H_Mean", "H_Std"],
    "S": ["S_Mean", "S_Std"],
    "V": ["V_Mean", "V_Std"],
    "R": ["R_Mean", "R_Std"],
    "G": ["G_Mean", "G_Std"],
    "B": ["B_Mean", "B_Std"],
    # "Day": ["Day"],
}

models = {
    "Linear Regression": LinearRegression(),
    "Ridge Regression": Ridge(alpha=1.0),
    "Lasso Regression": Lasso(alpha=0.1),
    "Polynomial Regression": make_pipeline(PolynomialFeatures(degree=2), LinearRegression()),
    "Elastic Net": ElasticNet(alpha=0.1, l1_ratio=0.5),
    "Huber Regressor": HuberRegressor(),
    "Quantile Regressor": QuantileRegressor(quantile=0.5, alpha=0.1),
    "Random Forest": RandomForestRegressor(n_estimators=100, random_state=42),
    "XGBoost": XGBRegressor(n_estimators=200, learning_rate=0.05, max_depth=6, random_state=42),
    "LightGBM": LGBMRegressor(n_estimators=200, learning_rate=0.05, max_depth=6, random_state=42)
}

In [10]:
def progressive_features(features_dict):
    feature_combinations = {}

    feature_groups = list(features_dict.keys())  # ["L", "a", "b", "H", ...]
    
    for i in range(1, len(feature_groups) + 1):
        for comb in combinations(feature_groups, i):  # Create feature group combinations
            combined_columns = sum([features_dict[key] for key in comb], [])  # Map to actual column names
            feature_combinations[",".join(comb)] = combined_columns

    print(f"Generated {len(feature_combinations)} feature combinations.")
    return feature_combinations

In [11]:
def mix_orange(df, red_weight=0.8, green_weight=0.2):
    """
    Create a new feature that represents a mixed color (Orange) from Red and Green.
    
    This function computes both:
    - Mixed color (Mean values)
    - Mixed color (Standard Deviation)
    - Converts the mixed color to LAB and HSV
    """
    # Compute the mixed color mean
    df["Mixed_RedGreen_Mean"] = (df["R_Mean"] * red_weight + df["G_Mean"] * green_weight).astype(int)

    # Compute the mixed color standard deviation
    df["Mixed_RedGreen_Std"] = (df["R_Std"] * red_weight + df["G_Std"] * green_weight).astype(int)

    # Convert Mixed RGB to LAB & HSV
    mixed_rgb = np.stack([df["Mixed_RedGreen_Mean"], df["Mixed_RedGreen_Mean"], np.zeros_like(df["Mixed_RedGreen_Mean"])], axis=1)
    mixed_bgr = np.array(mixed_rgb, dtype=np.uint8)[:, np.newaxis, :]  # Convert to BGR format
    
    mixed_lab = cv2.cvtColor(mixed_bgr, cv2.COLOR_RGB2LAB)[:, 0, :]
    mixed_hsv = cv2.cvtColor(mixed_bgr, cv2.COLOR_RGB2HSV)[:, 0, :]

    # Add Mean Values
    df["Mixed_L_Mean"] = mixed_lab[:, 0]
    df["Mixed_a_Mean"] = mixed_lab[:, 1]
    df["Mixed_b_Mean"] = mixed_lab[:, 2]

    df["Mixed_H_Mean"] = mixed_hsv[:, 0]
    df["Mixed_S_Mean"] = mixed_hsv[:, 1]
    df["Mixed_V_Mean"] = mixed_hsv[:, 2]

    # Add Standard Deviation (Since we're mixing, we use an estimated std based on weighted input stds)
    df["Mixed_L_Std"] = (df["L_Std"] * red_weight + df["L_Std"] * green_weight).astype(int)
    df["Mixed_a_Std"] = (df["a_Std"] * red_weight + df["a_Std"] * green_weight).astype(int)
    df["Mixed_b_Std"] = (df["b_Std"] * red_weight + df["b_Std"] * green_weight).astype(int)

    df["Mixed_H_Std"] = (df["H_Std"] * red_weight + df["H_Std"] * green_weight).astype(int)
    df["Mixed_S_Std"] = (df["S_Std"] * red_weight + df["S_Std"] * green_weight).astype(int)
    df["Mixed_V_Std"] = (df["V_Std"] * red_weight + df["V_Std"] * green_weight).astype(int)

    return df

base_features["Orange"] = ["Mixed_RedGreen_Mean", "Mixed_RedGreen_Std"]
base_features["Orange_Lab"] = ["Mixed_L_Mean", "Mixed_a_Mean", "Mixed_b_Mean", "Mixed_L_Std", "Mixed_a_Std", "Mixed_b_Std"]
base_features["Orange_HSV"] = ["Mixed_H_Mean", "Mixed_S_Mean", "Mixed_V_Mean", "Mixed_H_Std", "Mixed_S_Std", "Mixed_V_Std"]

In [12]:
def mix_yellow(df, red_weight=0.5, green_weight=0.5):
    """
    Create a new feature that represents a mixed color (Yellow) from Red and Green.
    
    This function computes:
    - Mixed color (Mean values)
    - Mixed color (Standard Deviation)
    - Converts the mixed color to LAB and HSV
    """
    # Compute the mixed color mean
    df["Mixed_RedGreenYellow_Mean"] = (df["R_Mean"] * red_weight + df["G_Mean"] * green_weight).astype(int)

    # Compute the mixed color standard deviation
    df["Mixed_RedGreenYellow_Std"] = (df["R_Std"] * red_weight + df["G_Std"] * green_weight).astype(int)

    # Convert Mixed RGB to LAB & HSV
    mixed_rgb = np.stack([df["Mixed_RedGreenYellow_Mean"], df["Mixed_RedGreenYellow_Mean"], np.zeros_like(df["Mixed_RedGreenYellow_Mean"])], axis=1)
    mixed_bgr = np.array(mixed_rgb, dtype=np.uint8)[:, np.newaxis, :]  # Convert to BGR format
    
    mixed_lab = cv2.cvtColor(mixed_bgr, cv2.COLOR_RGB2LAB)[:, 0, :]
    mixed_hsv = cv2.cvtColor(mixed_bgr, cv2.COLOR_RGB2HSV)[:, 0, :]

    # Add Mean Values
    df["Mixed_Yellow_L_Mean"] = mixed_lab[:, 0]
    df["Mixed_Yellow_a_Mean"] = mixed_lab[:, 1]
    df["Mixed_Yellow_b_Mean"] = mixed_lab[:, 2]

    df["Mixed_Yellow_H_Mean"] = mixed_hsv[:, 0]
    df["Mixed_Yellow_S_Mean"] = mixed_hsv[:, 1]
    df["Mixed_Yellow_V_Mean"] = mixed_hsv[:, 2]

    # Add Standard Deviation (Since we're mixing, we use an estimated std based on weighted input stds)
    df["Mixed_Yellow_L_Std"] = (df["L_Std"] * red_weight + df["L_Std"] * green_weight).astype(int)
    df["Mixed_Yellow_a_Std"] = (df["a_Std"] * red_weight + df["a_Std"] * green_weight).astype(int)
    df["Mixed_Yellow_b_Std"] = (df["b_Std"] * red_weight + df["b_Std"] * green_weight).astype(int)

    df["Mixed_Yellow_H_Std"] = (df["H_Std"] * red_weight + df["H_Std"] * green_weight).astype(int)
    df["Mixed_Yellow_S_Std"] = (df["S_Std"] * red_weight + df["S_Std"] * green_weight).astype(int)
    df["Mixed_Yellow_V_Std"] = (df["V_Std"] * red_weight + df["V_Std"] * green_weight).astype(int)

    return df
base_features["Yellow"] = ["Mixed_RedGreenYellow_Mean", "Mixed_RedGreenYellow_Std"]
base_features["Yellow_LAB"] = ["Mixed_Yellow_L_Mean", "Mixed_Yellow_a_Mean", "Mixed_Yellow_b_Mean", "Mixed_Yellow_L_Std", "Mixed_Yellow_a_Std", "Mixed_Yellow_b_Std"]
base_features["Yellow_HSV"] = ["Mixed_Yellow_H_Mean", "Mixed_Yellow_S_Mean", "Mixed_Yellow_V_Mean", "Mixed_Yellow_H_Std", "Mixed_Yellow_S_Std", "Mixed_Yellow_V_Std"]


In [13]:
def create_interaction_features(df):
    """
    Creates interaction features (cross-multiplication) to enhance model learning.
    """
    df["R_G_Interaction"] = df["R_Mean"] * df["G_Mean"]
    df["R_B_Interaction"] = df["R_Mean"] * df["B_Mean"]
    df["G_B_Interaction"] = df["G_Mean"] * df["B_Mean"]
    
    df["L_H_Interaction"] = df["L_Mean"] * df["H_Mean"]
    df["a_S_Interaction"] = df["a_Mean"] * df["S_Mean"]
    df["b_V_Interaction"] = df["b_Mean"] * df["V_Mean"]

    return df

In [14]:
def add_polynomial_features(X, degree=2):
    poly = PolynomialFeatures(degree=degree, include_bias=False)
    X_poly = poly.fit_transform(X)
    return pd.DataFrame(X_poly, columns=poly.get_feature_names_out(X.columns))


In [ ]:
# # 🔹 Train and Evaluate Models
# def train_and_evaluate(X, y, feature_sets, models, output_csv_path):
#     results = []

#     # Split dataset
#     X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

#     # ✅ Scale Data
#     scaler = StandardScaler()
#     X_train_scaled = scaler.fit_transform(X_train)
#     X_test_scaled = scaler.transform(X_test)

#     # Convert back to DataFrame
#     X_train = pd.DataFrame(X_train_scaled, columns=X_train.columns)
#     X_test = pd.DataFrame(X_test_scaled, columns=X_test.columns)

#     for feature_name, feature_list in feature_sets.items():
#         X_train_subset = X_train[feature_list]
#         X_test_subset = X_test[feature_list]

#         for model_name, model in models.items():
#             model.fit(X_train_subset, y_train)
#             y_pred = model.predict(X_test_subset)

#             mse = mean_squared_error(y_test, y_pred)
#             r2 = r2_score(y_test, y_pred)

#             # Store results
#             results.append({
#                 "Feature Set": feature_name,
#                 "Model": model_name,
#                 "MSE": mse,
#                 "R2 Score": r2
#             })

#             logging.info(f"Features: {feature_name}, Model: {model_name}, MSE: {mse:.4f}, R2 Score: {r2:.4f}")

#     # Save results
#     df_results = pd.DataFrame(results)
#     df_results.to_csv(output_csv_path, index=False)
#     print(f"Regression evaluation complete. Results saved to {output_csv_path}")

    from tqdm import tqdm
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import mean_squared_error, r2_score
import logging
import pandas as pd
from sklearn.linear_model import LinearRegression, Ridge, Lasso, ElasticNet, HuberRegressor, QuantileRegressor
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import PolynomialFeatures
from sklearn.ensemble import RandomForestRegressor
from xgboost import XGBRegressor
from lightgbm import LGBMRegressor

def train_and_evaluate(X, y, feature_sets, models, output_csv_path):
    results = []

    # Split dataset
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

    # ✅ Scale Data
    scaler = StandardScaler()
    X_train_scaled = scaler.fit_transform(X_train)
    X_test_scaled = scaler.transform(X_test)

    # Convert back to DataFrame
    X_train = pd.DataFrame(X_train_scaled, columns=X_train.columns)
    X_test = pd.DataFrame(X_test_scaled, columns=X_test.columns)

    # Track progress with tqdm
    total_steps = len(feature_sets) * len(models)  # Total steps in the iteration (feature sets * models)
    progress_bar = tqdm(total=total_steps, desc="Training Progress", unit="step")

    for feature_name, feature_list in feature_sets.items():
        X_train_subset = X_train[feature_list]
        X_test_subset = X_test[feature_list]

        for model_name, model in models.items():
            # Train and evaluate the model
            model.fit(X_train_subset, y_train)
            y_pred = model.predict(X_test_subset)

            mse = mean_squared_error(y_test, y_pred)
            r2 = r2_score(y_test, y_pred)

            # Store results
            results.append({
                "Feature Set": feature_name,
                "Model": model_name,
                "MSE": mse,
                "R2 Score": r2
            })

            logging.info(f"Features: {feature_name}, Model: {model_name}, MSE: {mse:.4f}, R2 Score: {r2:.4f}")

            # Update progress bar
            progress_bar.update(1)

    # Close the progress bar
    progress_bar.close()

    # Save results
    df_results = pd.DataFrame(results)
    df_results.to_csv(output_csv_path, index=False)
    print(f"Regression evaluation complete. Results saved to {output_csv_path}")


In [ ]:
# 🔹 Load Data
from multiprocessing import Pool


df = mix_orange(df)  # Apply color mixing
df = mix_yellow(df)
df = create_interaction_features(df)  # Add interaction features

# Define X (features) and y (target)
X = df[sum(base_features.values(), [])]
y = df["Weight"]

# Train & Evaluate Models
with Pool(processes=12) as pool:
    train_and_evaluate(X, y, progressive_features(base_features), models, "../output/train_csv/interact.csv")

2025-02-10 09:52:11,378 - INFO - Features: L, Model: Linear Regression, MSE: 109.9874, R2 Score: 0.5777
2025-02-10 09:52:11,381 - INFO - Features: L, Model: Ridge Regression, MSE: 109.9826, R2 Score: 0.5777
2025-02-10 09:52:11,383 - INFO - Features: L, Model: Lasso Regression, MSE: 110.3288, R2 Score: 0.5764
2025-02-10 09:52:11,386 - INFO - Features: L, Model: Polynomial Regression, MSE: 102.8202, R2 Score: 0.6052
2025-02-10 09:52:11,388 - INFO - Features: L, Model: Elastic Net, MSE: 110.4104, R2 Score: 0.5760
2025-02-10 09:52:11,397 - INFO - Features: L, Model: Huber Regressor, MSE: 119.1353, R2 Score: 0.5425
2025-02-10 09:52:11,540 - INFO - Features: L, Model: Quantile Regressor, MSE: 157.9552, R2 Score: 0.3935


Generated 32767 feature combinations.


2025-02-10 09:52:11,625 - INFO - Features: L, Model: Random Forest, MSE: 132.3010, R2 Score: 0.4920
2025-02-10 09:52:11,686 - INFO - Features: L, Model: XGBoost, MSE: 131.0228, R2 Score: 0.4969
2025-02-10 09:52:11,741 - INFO - Features: L, Model: LightGBM, MSE: 114.5511, R2 Score: 0.5601
2025-02-10 09:52:11,744 - INFO - Features: a, Model: Linear Regression, MSE: 204.3919, R2 Score: 0.2152
2025-02-10 09:52:11,746 - INFO - Features: a, Model: Ridge Regression, MSE: 201.2290, R2 Score: 0.2273
2025-02-10 09:52:11,749 - INFO - Features: a, Model: Lasso Regression, MSE: 200.1535, R2 Score: 0.2315
2025-02-10 09:52:11,754 - INFO - Features: a, Model: Polynomial Regression, MSE: 188.5678, R2 Score: 0.2759
2025-02-10 09:52:11,756 - INFO - Features: a, Model: Elastic Net, MSE: 194.3180, R2 Score: 0.2539
2025-02-10 09:52:11,761 - INFO - Features: a, Model: Huber Regressor, MSE: 199.5989, R2 Score: 0.2336
2025-02-10 09:52:11,898 - INFO - Features: a, Model: Quantile Regressor, MSE: 232.6116, R2 Sc

[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.000071 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 182
[LightGBM] [Info] Number of data points in the train set: 269, number of used features: 2
[LightGBM] [Info] Start training from score 119.898141
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -

2025-02-10 09:52:11,986 - INFO - Features: a, Model: Random Forest, MSE: 206.0168, R2 Score: 0.2089
2025-02-10 09:52:12,362 - INFO - Features: a, Model: XGBoost, MSE: 244.0735, R2 Score: 0.0628
2025-02-10 09:52:12,402 - INFO - Features: a, Model: LightGBM, MSE: 186.6173, R2 Score: 0.2834
2025-02-10 09:52:12,405 - INFO - Features: b, Model: Linear Regression, MSE: 103.4508, R2 Score: 0.6028
2025-02-10 09:52:12,411 - INFO - Features: b, Model: Ridge Regression, MSE: 100.3933, R2 Score: 0.6145
2025-02-10 09:52:12,413 - INFO - Features: b, Model: Lasso Regression, MSE: 100.8283, R2 Score: 0.6128
2025-02-10 09:52:12,417 - INFO - Features: b, Model: Polynomial Regression, MSE: 97.3940, R2 Score: 0.6260
2025-02-10 09:52:12,419 - INFO - Features: b, Model: Elastic Net, MSE: 141.9495, R2 Score: 0.4549
2025-02-10 09:52:12,423 - INFO - Features: b, Model: Huber Regressor, MSE: 103.8710, R2 Score: 0.6012
2025-02-10 09:52:12,525 - INFO - Features: b, Model: Quantile Regressor, MSE: 234.4939, R2 Sco

...

2025-02-10 09:52:36,164 - INFO - Features: S,G, Model: Random Forest, MSE: 103.6607, R2 Score: 0.6020
2025-02-10 09:52:36,250 - INFO - Features: S,G, Model: XGBoost, MSE: 136.2476, R2 Score: 0.4768
2025-02-10 09:52:36,280 - INFO - Features: S,G, Model: LightGBM, MSE: 113.1346, R2 Score: 0.5656
2025-02-10 09:52:36,285 - INFO - Features: S,B, Model: Linear Regression, MSE: 102.2040, R2 Score: 0.6076
2025-02-10 09:52:36,290 - INFO - Features: S,B, Model: Ridge Regression, MSE: 100.1781, R2 Score: 0.6153
2025-02-10 09:52:36,294 - INFO - Features: S,B, Model: Lasso Regression, MSE: 99.3844, R2 Score: 0.6184
2025-02-10 09:52:36,297 - INFO - Features: S,B, Model: Polynomial Regression, MSE: 115.4409, R2 Score: 0.5567
2025-02-10 09:52:36,300 - INFO - Features: S,B, Model: Elastic Net, MSE: 94.6620, R2 Score: 0.6365
2025-02-10 09:52:36,304 - INFO - Features: S,B, Model: Huber Regressor, MSE: 104.6091, R2 Score: 0.5983
2025-02-10 09:52:36,395 - INFO - Features: S,B, Model: Quantile Regressor, MS

[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.000044 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 363
[LightGBM] [Info] Number of data points in the train set: 269, number of used features: 4
[LightGBM] [Info] Start training from score 119.898141
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, b

2025-02-10 09:52:36,500 - INFO - Features: S,B, Model: Random Forest, MSE: 109.9229, R2 Score: 0.5779
2025-02-10 09:52:36,645 - INFO - Features: S,B, Model: XGBoost, MSE: 133.1221, R2 Score: 0.4888
2025-02-10 09:52:36,677 - INFO - Features: S,B, Model: LightGBM, MSE: 143.5345, R2 Score: 0.4489
2025-02-10 09:52:36,680 - INFO - Features: S,Orange, Model: Linear Regression, MSE: 100.6686, R2 Score: 0.6135
2025-02-10 09:52:36,682 - INFO - Features: S,Orange, Model: Ridge Regression, MSE: 98.8593, R2 Score: 0.6204
2025-02-10 09:52:36,684 - INFO - Features: S,Orange, Model: Lasso Regression, MSE: 97.5283, R2 Score: 0.6255
2025-02-10 09:52:36,687 - INFO - Features: S,Orange, Model: Polynomial Regression, MSE: 104.1948, R2 Score: 0.5999
2025-02-10 09:52:36,690 - INFO - Features: S,Orange, Model: Elastic Net, MSE: 94.4520, R2 Score: 0.6373
2025-02-10 09:52:36,694 - INFO - Features: S,Orange, Model: Huber Regressor, MSE: 101.6634, R2 Score: 0.6096
2025-02-10 09:52:36,786 - INFO - Features: S,Ora

[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.000034 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 362
[LightGBM] [Info] Number of data points in the train set: 269, number of used features: 4
[LightGBM] [Info] Start training from score 119.898141
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, b

KeyboardInterrupt: 